# 生成式对抗网络   Generative Adverasrial Network

## 1. 配置GPU 导入相关的库

In [1]:
from tensorflow.keras import models, layers, optimizers
from tensorflow import keras
import numpy as np
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
from tensorflow.compat.v1.keras.backend import set_session
from tensorflow.compat.v1 import ConfigProto, Session
config = ConfigProto(allow_soft_placement=True)
config.gpu_options.per_process_gpu_memory_fraction = 0.6
set_session(Session(config=config))

## 2. Generator  共计 62w个参数

In [2]:
latent_dim = 32     # 潜在可能性空间
height = 32         # 图片尺寸
width = 32          # 图片尺寸
channels = 3        # 图片的通道数

generator_input = keras.Input(shape=(latent_dim,))
# 类似于解码器
# 每层的激活函数采用LeakyReLU 相比于ReLU LeakReLU在输入为负数也有激活 缓解梯度的稀疏性
x = layers.Dense(128*16*16)(generator_input)
x = layers.LeakyReLU()(x)

x = layers.Reshape((16, 16, 128))(x)    # 重新塑性成 14x14x128 的数组 128个特征图
x = layers.Conv2D(256, (5, 5), padding='same')(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2DTranspose(256, (4, 4), padding='same', strides=(2, 2))(x)
x = layers.LeakyReLU()(x)
# 在此上采样为（28, 28, 256）的特征图

x = layers.Conv2D(256, (5, 5), padding='same')(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(256, (5, 5), padding='same')(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(channels, (7, 7), activation='tanh', padding='same')(x)

generator = keras.models.Model(generator_input, x)
generator.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32)]              0         
_________________________________________________________________
dense (Dense)                (None, 32768)             1081344   
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 32768)             0         
_________________________________________________________________
reshape (Reshape)            (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 16, 16, 256)       819456    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 16, 16, 256)       0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 32, 32, 256)       104883

## 3. Discriminator 判别器  共计8w 参数

In [3]:
discriminator_input = keras.Input(shape=(height, width, channels))

x = layers.Conv2D(128, (3, 3))(discriminator_input)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(128, (4, 4), strides=(2, 2))(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(128, (4, 4), strides=(2, 2))(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(128, (4, 4), strides=(2, 2))(x)
x = layers.LeakyReLU()(x)

x = layers.Flatten()(x)

x = layers.Dropout(0.4)(x)
x = layers.Dense(1, activation='sigmoid')(x)

discriminator = keras.models.Model(discriminator_input, x)
discriminator.summary()
discriminator_optimizer = optimizers.RMSprop(
    lr = 0.0008,
    decay = 1e-8,
    clipvalue = 1.0
)

discriminator.compile(
    optimizer = discriminator_optimizer,
    loss = 'binary_crossentropy'
)

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 30, 30, 128)       3584      
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 30, 30, 128)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 14, 14, 128)       262272    
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 6, 6, 128)         262272    
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 6, 6, 128)         0   

## 4. 生成对抗网络 Generative Adverasrial Network

In [4]:
discriminator.trainable = False     # 判别器不允许训练

gan_input = keras.Input(shape=(latent_dim,))
gan_output = discriminator(generator(gan_input))
gan = keras.models.Model(gan_input, gan_output)

gan_optimizer = optimizers.RMSprop(
    lr = 0.0004,
    decay = 1e-8,
    clipvalue = 1.0
)
gan.compile(optimizer=gan_optimizer, loss='binary_crossentropy')
gan.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 32)]              0         
_________________________________________________________________
model (Functional)           (None, 32, 32, 3)         6264579   
_________________________________________________________________
model_1 (Functional)         (None, 1)                 790913    
Total params: 7,055,492
Trainable params: 6,264,579
Non-trainable params: 790,913
_________________________________________________________________


## 5. 开始训练GAN

In [5]:
import os
from tensorflow.keras.preprocessing import image

(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

x_train = x_train[y_train.flatten() == 6]
x_train = x_train.reshape(
    (x_train.shape[0],)+(height, width, channels)).astype('float32') / 255.
iterations = 10000
batch_size = 32
save_dir = '/home/oneran/GAN_pictures/forg'
start = 0
for step in range(iterations):
    random_latent_vector = np.random.normal(size=(batch_size, latent_dim))
    
    generated_image = generator.predict(random_latent_vector)

    stop = start + batch_size
    real_image = x_train[start: stop]

    combined_images = np.concatenate([generated_image, real_image], axis=0)

    labels = np.concatenate([np.ones((batch_size, 1)), np.zeros((batch_size, 1))])
    labels += 0.05 * np.random.random(size=labels.shape)

    d_loss = discriminator.train_on_batch(combined_images, labels)
    
    random_latent_vector = np.random.normal(size=(batch_size, latent_dim))

    misleading_labels = np.zeros(shape=(batch_size, 1))

    a_loss = gan.train_on_batch(random_latent_vector, misleading_labels)
    
    start += batch_size
    if start > len(x_train) - batch_size:
        start = 0
    
    if step % 100 == 0:
        gan.save_weights('gan.h5')

        print('discriminator at loss:', d_loss)
        print('adversarial at loss:', a_loss)

        img = image.array_to_img(generated_image[0]*255., scale=False)
        img.save(os.path.join(save_dir, 'generated_'+str(step)+'.png'))

        img = image.array_to_img(real_image[0]*255., scale=False)
        img.save(os.path.join(save_dir, 'real_'+str(step)+'.png'))


discriminator at loss: 0.7008483409881592
adversarial at loss: 0.6835333704948425
discriminator at loss: 0.7034196853637695
adversarial at loss: 0.8704048991203308
discriminator at loss: 0.6908178925514221
adversarial at loss: 0.74298095703125
discriminator at loss: 0.689926028251648
adversarial at loss: 0.7600042819976807
discriminator at loss: 0.7033820152282715
adversarial at loss: 0.7470231056213379
discriminator at loss: 0.690680980682373
adversarial at loss: 0.7627683877944946
discriminator at loss: 0.6979295015335083
adversarial at loss: 0.7489864826202393
discriminator at loss: 0.7015222311019897
adversarial at loss: 0.7528306245803833
discriminator at loss: 0.6934388875961304
adversarial at loss: 0.7811828851699829
discriminator at loss: 0.6890732049942017
adversarial at loss: 0.7545453310012817
discriminator at loss: 0.7043299674987793
adversarial at loss: 0.7566009759902954
discriminator at loss: 0.6918659210205078
adversarial at loss: 0.7675271034240723
discriminator at los

In [6]:
discriminator.summary()
gan.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 30, 30, 128)       3584      
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 30, 30, 128)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 14, 14, 128)       262272    
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 6, 6, 128)         262272    
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 6, 6, 128)         0   